In [20]:
from utils import list_images, simulate_cytoplasm
from tqdm import tqdm
from cellpose import models, core, io
import re
import numpy as np
from tifffile import imread, imwrite
from skimage.color import rgb2gray
from collections import defaultdict
from pathlib import Path
from skimage.measure import regionprops_table
import napari
import pandas as pd

#Check if notebook has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose-SAM and Spotiflow models
model = models.CellposeModel(gpu=True)

In [18]:
data_folder = r"\\forskning.it.ntnu.no\ntnu\mh\ikom\cmic_konfokal\MariiaY\Evos Anna nanoparticles\LNPs.2025-11-05-10-56-18"

In [19]:
images = list_images(data_folder, format="tif")

In [22]:
# === YOUR INPUTS ===
files = images

output_dir = Path("./processed_tiffs")
output_dir.mkdir(parents=True, exist_ok=True)


# Regex to extract well, frame, channel
pattern = re.compile(r'.*_(?P<well>A\d{2})f(?P<frame>\d{2})d(?P<channel>\d)\.TIF$', re.IGNORECASE)

groups = defaultdict(dict)
for f in files:
    m = pattern.match(f)
    if m:
        groups[(m.group('well'), m.group('frame'))][int(m.group('channel'))] = f


def grayscale_skimage(img):
    # Case 1: TIFF loads as (3, H, W)
    if img.ndim == 3 and img.shape[0] == 3 and img.shape[-1] != 3:
        img = np.moveaxis(img, 0, -1)  # → (H, W, 3)
    
    # Case 2: Already (H, W, 3) → fine
    if img.ndim == 3 and img.shape[-1] == 3:
        gray01 = rgb2gray(img)  # float64 in [0,1]
    
    else:
        raise ValueError(f"Unexpected image shape {img.shape}")

    # Restore original dtype
    if np.issubdtype(img.dtype, np.integer):
        maxval = np.iinfo(img.dtype).max
        gray = (gray01 * maxval).astype(img.dtype)
    else:
        gray = gray01.astype(img.dtype)

    return gray


# Build & save stacks
for (well, frame), channels in tqdm(groups.items()):
    expected = [1, 2, 4]
    if not all(c in channels for c in expected):
        print(f"Skipping incomplete {well}f{frame}")
        continue

    slices = []
    for c in expected:
        rgb = imread(channels[c], series=None)
        gray = grayscale_skimage(rgb)
        slices.append(gray)

    stack = np.stack(slices, axis=0)
    out_path = output_dir / f"{well}f{frame}.tif"
    imwrite(out_path, stack)

    print("Saved:", out_path)


  0%|          | 0/48 [00:00<?, ?it/s]<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A01f00d1.TIF'> OME series is missing 2 frames. Missing data are zeroed
<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A01f00d2.TIF'> OME series is missing 2 frames. Missing data are zeroed
<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A01f00d4.TIF'> OME series is missing 2 frames. Missing data are zeroed
  2%|▏         | 1/48 [00:00<00:27,  1.74it/s]<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A01f01d1.TIF'> OME series is missing 2 frames. Missing data are zeroed
<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A01f01d2.TIF'> OME series is missing 2 frames. Missing data are zeroed
<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A01f01d4.TIF'> OME series is missing 2 frames. Missing data are zeroed


Saved: processed_tiffs\A01f00.tif


  4%|▍         | 2/48 [00:00<00:16,  2.73it/s]<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A01f02d1.TIF'> OME series is missing 2 frames. Missing data are zeroed
<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A01f02d2.TIF'> OME series is missing 2 frames. Missing data are zeroed
<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A01f02d4.TIF'> OME series is missing 2 frames. Missing data are zeroed


Saved: processed_tiffs\A01f01.tif


  6%|▋         | 3/48 [00:01<00:13,  3.30it/s]<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A01f03d1.TIF'> OME series is missing 2 frames. Missing data are zeroed
<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A01f03d2.TIF'> OME series is missing 2 frames. Missing data are zeroed


...

Saved: processed_tiffs\A04f09.tif


 98%|█████████▊| 47/48 [00:12<00:00,  3.58it/s]<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A04f11d1.TIF'> OME series is missing 2 frames. Missing data are zeroed
<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A04f11d2.TIF'> OME series is missing 2 frames. Missing data are zeroed
<tifffile.TiffFile 'LNPs_Plate_R_p00_0_A04f11d4.TIF'> OME series is missing 2 frames. Missing data are zeroed


Saved: processed_tiffs\A04f10.tif


100%|██████████| 48/48 [00:12<00:00,  3.75it/s]

Saved: processed_tiffs\A04f11.tif


In [2]:
processed_img_path = "./processed_tiffs"
processed_images = list_images(processed_img_path, format="tif")

In [30]:
img = imread(processed_images[14])

In [32]:
field_of_view = Path(processed_images[14]).stem.split("f")[1]
well_id = Path(processed_images[14]).stem.split("f")[0]

# Create a dictionary containing all image descriptors
descriptor_dict = {"well_id": well_id, "FOV": field_of_view}

print(field_of_view, well_id)

02 A02


In [33]:
# Predict nuclei labels using CellposeSAM and simulate cytosol using pyclesperanto
nuclei_labels, flows, styles = model.eval(img[1], niter=1000) # need to check the arguments
cytoplasm_labels = simulate_cytoplasm(nuclei_labels, dilation_radius=3)

In [34]:
# Visualize results in Napari
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img)
viewer.add_labels(nuclei_labels)
viewer.add_labels(cytoplasm_labels)

<Labels layer 'cytoplasm_labels' at 0x1fcda03aaa0>

In [35]:
props_list = []

# Extract nuclei avg_int
props = regionprops_table(label_image=nuclei_labels,
                        intensity_image=img[0],
                        properties=["label", "intensity_mean"])

# Convert to dataframe
props_df = pd.DataFrame(props)

# Rename intensity_mean column to indicate the specific image
props_df.rename(columns={"intensity_mean": f"nuclei_avg_int"}, inplace=True)

# Append each props_df to props_list
props_list.append(props_df)


# Extract cytoplasm avg_int
props = regionprops_table(label_image=cytoplasm_labels,
                        intensity_image=img[0],
                        properties=["label", "intensity_mean"])

# Convert to dataframe
props_df = pd.DataFrame(props)

# Rename intensity_mean column to indicate the specific image
props_df.rename(columns={"intensity_mean": f"cytoplasm_avg_int"}, inplace=True)

# Append each props_df to props_list
props_list.append(props_df)

In [39]:
# Initialize the df with the first df in the list
props_df = props_list[0]
# Start looping from the second df in the list
for df in props_list[1:]:
    props_df = props_df.merge(df, on="label")

# Add each key-value pair from descriptor_dict to props_df at the specified position
insertion_position = 0
for key, value in descriptor_dict.items():
    props_df.insert(insertion_position, key, value)
    insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

props_df

,well_id,FOV,label,nuclei_avg_int,cytoplasm_avg_int
0,A02,02,1,25.904762,29.419048
1,A02,02,2,50.257143,39.710526
2,A02,02,3,57.235294,56.281553
3,A02,02,4,28.111421,34.078431
4,A02,02,5,33.768080,38.184466
...,...,...,...,...,...
941,A02,02,942,28.256000,41.141414
942,A02,02,943,27.235537,37.383721
943,A02,02,944,25.098131,39.313953
944,A02,02,945,27.974359,31.573034
